In [1]:
import binance
import pandas as pd
from ipywidgets import widgets
import seaborn as sns
import datetime
from matplotlib import pyplot as plt
import time
import datetime
import os
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import random
# timestamp (in ms) to start from: Dec 15th 2018
start_point = '1544844000000'

# current time: Apr 25th 11:34 am
current_point = 1556184840000

# values to ignore in your balance
low_limit = .0001 

# reference exchange for BTC valuation
btcusd = 'BTCUSDT'

# depth of history for min, hrs, day, month
depth_min = '1000'
depth_hrs = '120'
depth_day = '60'
depth_mth = '5'

keys = pd.read_csv("binancekey.csv")
api_key = keys.loc[0,"api_key"]
api_secret = keys.loc[0,"api_secret"]
binance.set(api_key, api_secret)

### fonctions réutilisables

# build function that converts time stamp to standard date/time
def clean_date(histo, value):
    histo["closeTime"] = histo["closeTime"].apply(lambda x: datetime.datetime.fromtimestamp(x/1000))
    histo["openTime"] = histo["openTime"].apply(lambda x: datetime.datetime.fromtimestamp(x/1000))
    histo["xch"] = value
    return histo

# build function that collects history by minutes/day/hours/month. 
def build_histo(minutes, hours, days, months):
    Hminutes = clean_date(pd.DataFrame(binance.klines(btcusd,minutes,limit = depth_min)),btcusd)
    Hhours = clean_date(pd.DataFrame(binance.klines(btcusd,hours,limit = depth_hrs)), btcusd)
    Hdays = clean_date(pd.DataFrame(binance.klines(btcusd,days,limit = depth_day)), btcusd)
    Hmonths = clean_date(pd.DataFrame(binance.klines(btcusd,months,limit = depth_mth)), btcusd)
    return Hminutes, Hhours, Hdays, Hmonths

In [ ]:
# rebuild 1 complete files from various archives
files = os.listdir("data binance/")
files.sort()

data = pd.DataFrame()
for items in files: 
    temp = pd.read_csv('data binance/' + items, index_col = 0)
    data = data.append(temp, ignore_index = True)
data = clean_date(data,btcusd)
data.to_csv("klines_all.csv")
# reload all Klines from file
data = pd.read_csv('klines_all.csv', index_col = 0)
data = data.sort_values("openTime")
data = data.reindex()

In [ ]:
# feature engineering: categorize records indicating if they are highest or lowest of next x rows

for i in range(0,len(data) - 1000):
    
    data.loc[i,"lowest_of_10"] = data.loc[i:i+9,"low"].min() # capture lowest of next 10
    data.loc[i,"lowest_of_50"] = data.loc[i:i+49,"low"].min() # capture lowest of next 50
    data.loc[i,"lowest_of_100"] = data.loc[i:i+99,"low"].min() # capture lowest of next 100
    data.loc[i,"lowest_of_1000"] = data.loc[i:i+999,"low"].min() # capture lowest of next 1000

    data.loc[i,"highest_of_10"] = data.loc[i:i+9,"high"].max() # record is highest of next 10
    data.loc[i,"highest_of_50"] = data.loc[i:i+49,"high"].max() # record is highest of next 50
    data.loc[i,"highest_of_100"] = data.loc[i:i+99,"high"].max() # record is highest of next 100
    data.loc[i,"highest_of_1000"] = data.loc[i:i+999,"high"].max() # record is highest of next 1000

# progress monitoring        
    
    if i == 100:
        print("over 100",datetime.datetime.fromtimestamp(time.time()))       
    if i == 10000:
        print("over 10k",datetime.datetime.fromtimestamp(time.time()))
    if i == 50000:
        print("over 50k",datetime.datetime.fromtimestamp(time.time()))
    if i == 100000:
        print("over 100k",datetime.datetime.fromtimestamp(time.time()))
    if i == 150000:
        print('over 150k',datetime.datetime.fromtimestamp(time.time()))
        
data.to_csv("with_lowest_of_10.csv")    

In [ ]:
data = pd.read_csv("with_lowest_of_10.csv", index_col = 0)

In [ ]:
''' patch
data["lowest_of_1000"] = 0
data["highest_of_1000"] = 0

for i in range(0,len(data) - 1000): 
    if data.loc[i,"is_l100"] == 1:
        data.loc[i,"lowest_of_1000"] = data.loc[i:i+999,"low"].min() # capture lowest of next 1000
    if data.loc[i,"is_h100"] == 1:
        data.loc[i,"highest_of_1000"] = data.loc[i:i+999,"high"].max() # record is highest of next 1000
'''

In [ ]:
# enrichissement avec les pointeurs
data["is_l10"] = data.lowest_of_10 == data.low
data["is_l50"] = data.lowest_of_50 == data.low
data["is_l100"] = data.lowest_of_100 == data.low
data["is_l1000"] = data.lowest_of_1000 == data.low

data["is_h10"] = data.highest_of_10 == data.high
data["is_h50"] = data.highest_of_50 == data.high
data["is_h100"] = data.highest_of_100 == data.high
data["is_h1000"] = data.highest_of_1000 == data.high

data["is_l10"] = [int(r) for r in data["is_l10"]]
data["is_l50"] = [int(r) for r in data["is_l50"]]
data["is_l100"] = [int(r) for r in data["is_l100"]]
data["is_l1000"] = [int(r) for r in data["is_l1000"]]

data["is_h10"] = [int(r) for r in data["is_h10"]]
data["is_h50"] = [int(r) for r in data["is_h50"]]
data["is_h100"] = [int(r) for r in data["is_h100"]]
data["is_h1000"] = [int(r) for r in data["is_h1000"]]

data.to_csv("with_lowest_of_10.csv") 


In [13]:
data = pd.read_csv("with_lowest_of_10.csv", index_col = 0)

,close,closeTime,high,low,numTrades,open,openTime,quoteVolume,volume,xch,...,is_l10,is_l50,is_l100,is_h10,is_h50,is_h100,highest_of_1000,lowest_of_1000,is_h1000,is_l1000
0,3251.54,2018-12-15 04:20:59.999,3252.77,3250.27,89,3250.27,2018-12-15 04:20:00,27264.262578,8.385634,BTCUSDT,...,0,0,0,0,0,0,0.0,0.0,0,0
1,3251.66,2018-12-15 04:21:59.999,3253.72,3250.41,105,3251.54,2018-12-15 04:21:00,36756.847331,11.303379,BTCUSDT,...,0,0,0,1,0,0,0.0,0.0,0,0
2,3248.80,2018-12-15 04:22:59.999,3251.16,3248.79,121,3250.79,2018-12-15 04:22:00,45509.798550,14.002981,BTCUSDT,...,0,0,0,1,0,0,0.0,0.0,0,0
3,3248.47,2018-12-15 04:23:59.999,3249.92,3247.83,114,3249.43,2018-12-15 04:23:00,48525.647730,14.937206,BTCUSDT,...,0,0,0,1,0,0,0.0,0.0,0,0
4,3247.37,2018-12-15 04:24:59.999,3249.37,3246.60,105,3247.81,2018-12-15 04:24:00,43017.224148,13.243858,BTCUSDT,...,0,0,0,0,0,0,0.0,0.0,0,0


In [24]:
# remains to do:
# try with is_l10/50/100 as input variable rather than output
# eliminate islxxx from latest records of each training set because in reality we won't know
# try and normalize each training set individually to keep only short term contextual behabiour
# try and tag each line with a realationship to MA50, 100, etc...

data = pd.read_csv("with_lowest_of_10.csv", index_col = 0)
data = data.loc[:len(data)-1000,:]# eliminate last 1000 records which are not enriched

data["gap"] = data["close"] - data["open"]
data["overall_gap"] = data["high"] - data["low"]
data["where_in_gap"] = data["close"] - (data["high"] + data["low"])/2

#X_columns = ["gap", "overall_gap", "where_in_gap", "numTrades"]
X_columns = ["close"]

y_columns = ['is_l10', 'is_l50','is_l100','is_l1000', 'is_h10','is_h50', 'is_h100','is_h1000']

time_step= 1000 # how many time units used to explain each output

#X = np.array(data.loc[:, X_columns])
#y = np.array(data.loc[:, y_columns])
X = data.loc[:, X_columns]
y = data.loc[:, y_columns]

dim_0 = X.shape[0] # final training set 
dim_1 = X.shape[1] # how many dimension each record has
dim_2 = y.shape[1] # how many output neurons 


train_lim = int(dim_0 *0.7) # training set length
val_lim = int(dim_0*0.85)  # validation set length

min_max_scaler = MinMaxScaler()
fitted = min_max_scaler.fit(X[:val_lim]) # don't fit on test data
X = fitted.transform(X) 
X = pd.DataFrame(X, columns = X_columns)

In [25]:
check_steps = [1000,5000,50000,100000, 150000]    

#X_temp = np.zeros((dim_0, time_step, dim_1))
X_temp = []
# pay attention, setting limit below should not go over total data length -1000 (no data enrichment beyond)

''' too early for conversion to array 
for i in range(time_step,dim_0):
    X_temp[i] = X[i-time_step:i]    
# progress monitoring        
    if i in check_steps:
        print("over {}".format(i),datetime.datetime.fromtimestamp(time.time()))       
#X_10 = X_temp[r[0] == 1 for r in y]
'''
# build l10/H10 training kit

y_temp_10 = []
y_temp_50 = []
y_temp_100 = []
y_temp_1000 = []
y_temp_other = []

X_temp_10 = [] # col 0 or 4
X_temp_50 = [] # col 1 or 5
X_temp_100 = [] # col 2 or 6
X_temp_1000 = [] # col 3 or 7
X_temp_other = [] # other

for i in range(time_step,dim_0):
#    X_temp.append(X[i-time_step:i].values.tolist())
#    if y["is_l10"][i] + y["is_h10"][i] > 0:
#        X_temp_10.append(X[i-time_step:i].values.tolist())
#        y_temp_10.append(y.iloc[i,[0,4]].values.tolist())
#        if y["is_l50"][i] + y["is_h50"][i] > 0:
#            X_temp_50.append(X[i-time_step:i].values.tolist())
#            y_temp_50.append(y.iloc[i,[1,5]].values.tolist())
    if y["is_l100"][i] + y["is_h100"][i] > 0:
        X_temp_100.append(X[i-time_step:i].values)
        y_temp_100.append(y.iloc[i,[2,6]].values)
        if y["is_l1000"][i] + y["is_h1000"][i] >0:
            X_temp_1000.append(X[i-time_step:i].values)
            y_temp_1000.append(y.iloc[i,[3,7]].values)
#    else:
#        X_temp_other.append(X[i-time_step:i].values.tolist())
#        y_temp_other.append(y.iloc[i,[0,4]].values.tolist())

# progress monitoring        
    if i in check_steps:
        print("over {}".format(i),datetime.datetime.fromtimestamp(time.time()))       
#X_10 = X_temp[r[0] == 1 for r in y]

over 1000 2019-05-09 15:30:26.542512
over 5000 2019-05-09 15:30:27.124286
over 50000 2019-05-09 15:30:32.209716
over 100000 2019-05-09 15:30:37.095945
over 150000 2019-05-09 15:30:42.438605


In [26]:
X_temp_other = []
y_temp_other = []
samp100_ind = X[y["is_l100"] + y["is_h100"] == 0].sample(len(y_temp_100)).index
for i in samp100_ind:
    X_temp_other.append(X[i-time_step:i].values)
    y_temp_other.append(y.iloc[i,[3,7]].values)

X_100 = X_temp_100 + X_temp_other
y_100 = y_temp_100 + y_temp_other

X_temp_other = []
y_temp_other = []
samp1000_ind = X[y["is_l1000"] + y["is_h1000"] == 0].sample(len(y_temp_1000)).index
for i in samp1000_ind:
    X_temp_other.append(X[i-time_step:i].values)
    y_temp_other.append(y.iloc[i,[3,7]].values)

X_1000 = X_temp_1000 + X_temp_other
y_1000 = y_temp_1000 + y_temp_other


In [31]:
def array_conv(file_x, file_y):
    dim1 = len(file_x)
    dim2 = len(file_x[0])
    dim3 = len(file_x[0][0])
    dim4 = len(file_y[0])
#   result_x = np.zeros((dim1, dim2, dim3))
    result_x = np.zeros((dim1, dim2))
    result_y = np.zeros((dim1,dim4))
    
    cpt_error = 0
    
    print(dim1,dim2,dim3)    
    
    for a in range(dim1-1):
        if a in check_steps:
            print("over {}".format(a),datetime.datetime.fromtimestamp(time.time()))       
        for b in range(dim2-1):

# specific for CNN preparation
            try:
                result_x[a][b] = np.array(file_x[a][b])
            except:
                result_x[a][b] = 0
                cpt_error += 1
    return result_x, result_y
'''
            for c in range(dim3-1):            
                try:
                    result_x[a][b][c] = np.array(file_x[a][b][c])
                except:
                    result_x[a][b][c] = 0
                    cpt_error += 1
        result_y[a] = np.array(file_y[a])
'''
            

    
def arr_to_files(X_inp,y_inp, pref):   
    temp_df = pd.DataFrame()
    temp_df['x'] = [r.tolist() for r in X_inp]
    temp_df['y'] = [r.tolist() for r in y_inp]
    temp_df = temp_df.sample(frac = 1)
    x_inp = temp_df['x'].values
    y_inp = temp_df['y'].values

    #    del_line = len(X_temp)%batch
   
    train_lim = int(len(X_inp)*0.7) # training set length
    val_lim = int(len(X_inp)*0.85)  # validation set length
    
    X_inp, y_inp = array_conv(X_inp, y_inp)
    
    print(X_inp.shape, train_lim,val_lim)
    
    X_train = X_inp[0:train_lim]
    X_val = X_inp[train_lim:val_lim]
    X_test = X_inp[val_lim:]
    y_train = y_inp[0:train_lim]
    y_val =  y_inp[train_lim:val_lim]
    y_test =  y_inp[val_lim:]
    
#    print(pref,'\n', X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape)
#    return X_train, X_val, X_test, y_train, y_val, y_test
    np.save(pref+'x_train',X_train)
    np.save(pref+'x_val',X_val)
    np.save(pref+'x_test',X_test)
    np.save(pref+'y_train',y_train)
    np.save(pref+'y_val',y_val)
    np.save(pref+'y_test',y_test)
#    return X_train, y_train

#arr_to_files(X_100,y_100,'traintest/100_')
arr_to_files(X_1000,y_1000,'traintest/1000_')

12800 1000 1
over 1000 2019-05-09 15:34:45.772455
over 5000 2019-05-09 15:34:49.407311
(12800, 1000) 8960 10880


NameError: name 'X_train' is not defined